# Containers

## Building Containers with DockerFile and Cloud Build

In [ ]:
%%writefile quickstart.sh
#!/bin/sh
echo "Hello, world! The time is $(date)." 

In [ ]:
%%writefile Dockerfile
FROM alpine
COPY quickstart.sh /
CMD ["/quickstart.sh"]

In [ ]:
%%bash
chmod +x quickstart.sh
gcloud builds submit --tag gcr.io/${GOOGLE_CLOUD_PROJECT}/quickstart-image .

## Building Containers with a build configuration file and Cloud Build

In [ ]:
%%writefile cloudbuild.yaml
steps:
- name: 'gcr.io/cloud-builders/docker'
  args: [ 'build', '-t', 'gcr.io/$PROJECT_ID/quickstart-image', '.' ]
images:
- 'gcr.io/$PROJECT_ID/quickstart-image'

In [ ]:
%%bash
gcloud builds submit --config cloudbuild.yaml .

## Building and Testing Containers with a build configuration file and Cloud Build 

In [ ]:
%%writefile quickstart.sh
#!/bin/sh
if [ -z "$1" ]
then
  echo "Hello, world! The time is $(date)."
  exit 0
else
  exit 1
fi

In [ ]:
%%writefile cloudbuild.yaml
steps:
- name: 'gcr.io/cloud-builders/docker'
  args: [ 'build', '-t', 'gcr.io/$PROJECT_ID/quickstart-image', '.' ]
- name: 'gcr.io/$PROJECT_ID/quickstart-image'
  args: ['fail']
images:
- 'gcr.io/$PROJECT_ID/quickstart-image

In [ ]:
%%bash
gcloud builds submit --config cloudbuild.yaml .

# Kubernetes

## Google Kubernetes Engine Deployments

In [ ]:
# set the environment variable for the zone and cluster name.
export my_zone=us-central1-a
export my_cluster=standard-cluster-1

# Configure kubectl tab completion
source <(kubectl completion bash)

# configure access to cluster
gcloud container clusters get-credentials $my_cluster --zone $my_zone

In [ ]:
%%writefile nginx-deployment.yaml
# create a deployment using deployment manifest
apiVersion: apps/v1
kind: Deployment
metadata:
  name: nginx-deployment
  labels:
    app: nginx
spec:
  replicas: 3
  selector:
    matchLabels:
      app: nginx
  template:
    metadata:
      labels:
        app: nginx
    spec:
      containers:
      - name: nginx
        image: nginx:1.7.9
        ports:
        - containerPort: 80

In [ ]:
# deploy manifest
kubectl apply -f ./nginx-deployment.yaml

kubectl get deployments

# scale the Pod
kubectl scale --replicas=3 deployment nginx-deployment

# update the version of nginx
kubectl set image deployment.v1.apps/nginx-deployment nginx=nginx:1.9.1 --record

# rollout status
kubectl rollout status deployment.v1.apps/nginx-deployment

# View the rollout history
kubectl rollout history deployment nginx-deployment

# roll back to the previous version
kubectl rollout undo deployments nginx-deployment

# View the details of the latest deployment revision
kubectl rollout history deployment/nginx-deployment --revision=3

In [ ]:
%%writefile service-nginx.yaml
# deploys a LoadBalancer service
apiVersion: v1
kind: Service
metadata:
  name: nginx
spec:
  type: LoadBalancer
  selector:
    app: nginx
  ports:
  - protocol: TCP
    port: 60000
    targetPort: 80

In [ ]:
# deploy service
kubectl apply -f ./service-nginx.yaml

# Verify the LoadBalancer creation
kubectl get service nginx

In [ ]:
%%writefile nginx-canary.yaml
# canary deployment directs a subset of users to the canary version
apiVersion: apps/v1
kind: Deployment
metadata:
  name: nginx-canary
  labels:
    app: nginx
spec:
  replicas: 1
  selector:
    matchLabels:
      app: nginx
  template:
    metadata:
      labels:
        app: nginx
        track: canary
        Version: 1.9.1
    spec:
      containers:
      - name: nginx
        image: nginx:1.9.1
        ports:
        - containerPort: 80

In [ ]:
# Create the canary deployment
kubectl apply -f nginx-canary.yaml

# verify
kubectl get deployments

# scale down the primary deployment to 0 replicas
kubectl scale --replicas=0 deployment nginx-deployment

In [ ]:
%%writefile service-nginx-canary.yaml
# Session affinity
apiVersion: v1
kind: Service
metadata:
  name: nginx
spec:
  type: LoadBalancer
  sessionAffinity: ClientIP
  selector:
    app: nginx
  ports:
  - protocol: TCP
    port: 60000
    targetPort: 80